# SIA Assignment 3: Characterising the orbits and masses of extrasolar planets

In this assignment, you will work with the same 5-hour long observation of HR 8799bcde from the week 6 workshop. Now we are going to go beyond detecting the 4 planets, to extracting astrometry and photometry for each planet.  

The two image analysis steps that you will be marked on (20% each, with an additional 30% of the mark based on code efficiency, structure, and comments) are: 

1) to measure the on-sky separation between each planet and star in units of astronomical units (AU) and compare to solar system planets
2) to measure the brightness of each planet and use exoplanet models to estimate the mass of each planet.

In addition, up to 30% of the mark on this assignment is based on evidence for version control, which is especially important when working as a group.  Thus, please use the last cell in this notebook (in markdown mode) to keep a log of changes as you work on this assignment.  

In [ ]:
# import a number of critical python packages
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch, ImageNormalize
from scipy import ndimage
import scipy.linalg as la
import matplotlib.ticker as ticker
from tqdm import tqdm
from astropy.table import Table
from scipy import interpolate

# python packages for astrometry / photometry
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, CircularAnnulus
from photutils.aperture import ApertureStats
from photutils.aperture import aperture_photometry

Some useful functions to implement PCA from the workshop:

In [ ]:
def calc_KL_basis(covar_psfs, ref_psfs_mean_sub, numbasis=200):

    # Given a covariance matrix built from an array of reference PSFs, calculate the eigenvectors, eigenvectors, and resulting K-L basis set.
    # Adapted from: https://pyklip.readthedocs.io/en/latest/
    max_basis = np.max(numbasis) + 1
    tot_basis = covar_psfs.shape[0]

    evals, evecs = la.eigh(covar_psfs, subset_by_index=[tot_basis-max_basis, tot_basis-1])

    # scipy.linalg.eigh spits out the eigenvalues/vectors smallest first so we need to reverse
    # we're going to recopy them to hopefully improve caching when doing matrix multiplication
    evals = np.copy(evals[::-1])
    evecs = np.copy(evecs[:,::-1], order='F') #fortran order to improve memory caching in matrix multiplication

    # calculate the KL basis vectors
    kl_basis = np.dot(ref_psfs_mean_sub.T, evecs)

    # multiply by 1 / sqrt(eigenvalues)    
    kl_basis = kl_basis * (1. / np.sqrt(evals * (kl_basis.shape[0] - 1)))[None, :]  #multiply a value for each row

    return(kl_basis)            

In [ ]:
def make_KLIP_psf(im, kl_basis, numbasis=200):

    # Given a science image and K-L basis set, project the science image onto the K-L basis set.
    # Adapted from: https://pyklip.readthedocs.io/en/latest/
    
    max_basis = np.max(numbasis) + 1
    sci = np.ravel(im) # flatten image into 1d array
    sci_mean_sub = sci - np.nanmean(sci) # subtract off mean

    sci_mean_sub_rows = np.tile(sci_mean_sub, (max_basis, 1))

    inner_products = np.dot(sci_mean_sub_rows, kl_basis)
                            #np.require(kl_basis, requirements=['F']))
    # select the KLIP modes we want for each level of KLIP by multiplying by lower diagonal matrix
    lower_tri = np.tril(np.ones([max_basis, max_basis]))
    inner_products = inner_products * lower_tri

    klip_psf = np.dot(inner_products[numbasis,:], kl_basis.T)
    klip_psf_2d = np.reshape(klip_psf, (im.shape[0], im.shape[1]))

    return(klip_psf_2d)

Open the data files:

In [ ]:
# open the reduced and centered SPHERE data cube
imname = 'ird_convert_recenter_dc5-IRD_SCIENCE_REDUCED_MASTER_CUBE-center_im_ext1_smallcube.fits'

# open the parallactic angle file:
# For this ADI sequence, the derotator was turned off, so the field-of-view will appear to rotate on the sky as the star 
# that is being observed rises and sets.
# The parallactic angle is the angle between North and the vertical axis of each of the images in the SPHERE data cube.

parname = 'ird_convert_recenter_dc5-IRD_SCIENCE_PARA_ROTATION_CUBE-rotnth.fits'

# get data
imcube = fits.getdata(imname)
pararr = fits.getdata(parname)


## Preamble: Run PCA with varying numbers of principal components to decide how many principal components to adopt

Let's start by running the PCA algorithm on the data with a varying number of principal components, to determine the effect of the number of principal components adopted on the final output image.

Since we will likely run this a few times, let's write some convenience functions to make things easier.  First, let's write a function to take an input data cube and make the corresponding covariance matrix and set of reference psfs.

In [ ]:
def make_cov_matrix(cube):
    # reshape array of psfs


    # subtract off the mean from each row of the ref_psfs array


    # find the covariance matrix of the array of reference images


    return(covar_psfs, ref_psfs_mean_sub)

Starting with the covariance matrix, reference psfs, image cube, and parallactic angle array, write a convenience function to calculate the KL basis set, build and subtract a PSF model for each image in the cube, then derotate and stack the final subtracted cube.  

In [ ]:
def run_full_PCA(covar_psfs, ref_psfs_mean_sub, data, pararr, numbasis=200):
    # calculate the KL basis vectors 
 
    # step through each image, make its own klip_psf, subtract from the on-sky data cube, derotate, and stack

    for i in tqdm(np.arange(datashape[0])):
        

    return np.nanmedian(, axis=0)    

Now run the PCA algorithm on the data cube with 5 to 55 principal components, in steps of 10 components.  Save the output to a data cube.

In [ ]:
# run PCA algorithm with 5 to 50 principal components, in steps of 5 components, save output in a datacube

covar_psfs, ref_psfs_mean_sub = make_cov_matrix(imcube)

ncomps = np.arange(5, 60, 10)

imstack = np.zeros((len(ncomps), imcube.shape[1], imcube.shape[2]))

for i in np.arange(len(ncomps)):


Plot the results.  Be sure to label how many principal components were used in each output image.

In [ ]:
#plot results

fig = plt.figure(figsize=(30,20))

for i in np.arange(len(ncomps)):

    ax = fig.add_subplot(2,3,i+1)



A few observations: 
1) the more principal components used, the better subtracted the speckle noise, but the longer the PCA takes to run.
2) the exoplanets don't look like perfect point sources -- they are super-imposed on a dark, angular streak.  This is the result of having light from the exoplanet in the stack of reference images used to build the KL-basis.  The dark streaks are due to self-subtraction of the planet from itself.  The more principal components used, the worse the self-subtraction will be.

Thus, choosing the best number of principal components to adopt is a balance between sufficient speckle suppression and minimising self-subtraction of each planet.  Let's use 35 principal components for the rest of this assignment, as this seems to provide a good balance between the two. 

## Image Analysis Step 1: find the on-sky separation between the planets and the star, compare with solar system planets

Like in workshop 4, we will use DAOStarFinder to find the positions of each planet.  We'll use the stacked image with 35 principal components subtracted, but you should find similar results with any of the stacked images from the last step.  Since you may want to run this step a few times, let's start by writing a convenience function to run DAOStarFinder and produce a table of sources:

In [ ]:
def run_DAOStarFinder(im, fwhm, threshold):

    # use sigma_clipped_stats from astropy to calculate the mean median and std

    # run DAOStarFinder


    return sources    

Let's start by running with fwhm = 4 pixels and a threshold of 100 times the standard deviation of the image.

In [ ]:
sources = run_DAOStarFinder(imstack[3, :, :], 4., 100.)

Print out your resulting source table

In [ ]:
sources

Overplot the detected sources on the image

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)

In [ ]:
fig = plt.figure(figsize=(20,20))

ax = fig.add_subplot(111)

norm = ImageNormalize(imstack[3, :, :], vmin=-5, vmax=5)
ax.imshow(imstack[3, :, :], norm=norm, origin='lower')

apertures.plot(color='white', lw=3.0, alpha=0.5)

All 4 planets are detected, but so is a lot of other residual speckle noise left over in the core of the image.  Adjust fwhm and threshold until you robustly detect only the 4 planets.  Print out the resulting source table and replot the image with the detections of the exoplanets overlaid.

In [ ]:
bestfwhm = 
bestthreshold = 
sources = run_DAOStarFinder(imstack[3, :, :], bestfwhm, bestthreshold)

In [ ]:
sources

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)

In [ ]:
fig = plt.figure(figsize=(20,20))

ax = fig.add_subplot(111)

norm = ImageNormalize(imstack[3, :, :], vmin=-5, vmax=5)
ax.imshow(imstack[3, :, :], norm=norm, origin='lower')

apertures.plot(color='white', lw=3.0, alpha=0.5)

## Find position of planets in polar coordinates, i.e. radius, position angle (theta) units 

The centre of the image is at pixel 200, 200.  The convention for angles in astronomy place 0 degrees at North, with angles increasing positively anticlockwise from North.

In [ ]:
xcen = 200.
ycen = 200.
r = np.sqrt(np.power(sources['xcentroid'] - xcen,2.) + np.power(sources['ycentroid'] - ycen,2.))
theta = np.arctan2(-(sources['xcentroid'] - xcen), sources['ycentroid'] - ycen)

Convert and print out your planet separations from units of pixels to units of arcseconds.  The pixel scale of the SPHERE detector is: 0.01225 arcsecond / pixel.

Now convert to units of astronomical units (AU).  1 AU is the average distance between the Earth and the Sun.  The distance to the HR 8799 system is 40.8 pc.  To convert to AU, you just need to multiply the separation in arcsec with the distance to the star in pc.

How do these values compare with the semi-major axes of the orbits of gas giants like Jupiter and Saturn in our own solar system?

Print out the position angles of each planet in degrees as well:

## Image Analysis Step 2: Use simulated exoplanets to estimate the brightness and mass each of the planets

Since the PCA algorithm is not flux-conserving, we lose some (unquantified) amount of brightness from each planet when we run PCA.  Thus, unlike the seeing-limited image used in Workshop 4, we cannot simply perform aperture photometry to estimate the brightness of each planet.  Instead, we will use an image of the star taken without the coronagraph to simulate exoplanets in the raw data, then run PCA, and compare the brightness (after self-subtraction) of the simulated planets to the actual planets.  Let's start by inspecting the stellar image we will use to simulate the planets:

In [ ]:
stellarim = fits.getdata('stellarim.fits')

In [ ]:
fig = plt.figure(figsize=(20,20))

ax = fig.add_subplot(111)

ax.imshow(stellarim, origin='lower')

This is an image of the star without the coronagraph taken right before the coronagraphic sequence you have been working with.  The original exposure time of the image was quite short to prevent saturation (in other words, hitting the brightness limit where the detector runs out of dynamic range), but the image was then scaled to the exposure time used for the coronagraphic sequence.  Now let's write another convenience function, to make a cube of simulated planet images at the correct position angle for each image in the sequence.  It should take the data cube, stellar image, and array of parallactic angles as input, and simulate an exoplanet at a radius of rad pixels, with position angle theta, and at some delta(magnitude) dmag fainter than the primary star.  

In [ ]:
def simulate_object(data, stellarim, pararr, rad, theta, dmag, xcen=200, ycen=200, imdiam=400):
    # goal of function: make an image cube with just simulated objects

    # make an array with the same dimensions as the inputted data cube, but filled with only zeros.  Hint: try using np.zeros
    psfcube =     

    # put small stellar image into the center of an image with the same shape as each slice of the data cube, but composed only of zeros. 
    psfpadded = 
    
    # figure out the appropriate position of the simulated object 
    xshifts = -rad * np.sin((pararr + theta) * (np.pi/180.))
    yshifts = rad * np.cos((pararr + theta) * (np.pi/180.))
    
    # for each slice in the array with the same dimensions as the inputted cube, insert the simulated planet image at the right position
    for i in tqdm(np.arange(data.shape[0])):
        psfcube[i,:,:] = ndimage.shift()
        
    # Scale simulated planet images to the input Delta(mag) value
    psfcube = psfcube * 10**(-dmag / 2.5)
    
    return(psfcube)


Simulate a test object 4 magnitudes fainter than the star, at a radius of 100 pixels and a position angle of 60 degrees.

In [ ]:
simcube = simulate_object(data, stellarim, pararr, 100., 60., 4.0)

Plot a few frames from the resulting cube to see what this looks like:

In [ ]:
#plot results

fig = plt.figure(figsize=(30,20))

for i in np.arange(6):

    ax = fig.add_subplot(2,3,i+1)

    norm = ImageNormalize(simcube[(i+1)*100, :, :], vmin=-5, vmax=5)
    ax.imshow(simcube[(i+1)*100, :, :], origin='lower')
    ax.set_title('frame '+str((i+1)*10), fontsize=20)

Now add your simulated object cube to the image cube and rerun PCA.  You'll need to remake the covariance matrix with the new data cube that includes the simulated object.  In fact, go ahead and write a convenience function to build the covariance matrix.

Add the simulated object to the data cube and rerun the PCA

In [ ]:
# add simulated object to data cube
datawithsimobject = data + simcube

# rerun PCA with 35 principal components.  You'll need to remake the covariance matrix in this case.


Display the resulting image.  You've simulated a very bright object, so you shouldn't see the planets.

In [ ]:
# plot result
fig = plt.figure(figsize=(20,20))

ax = fig.add_subplot(111)

ax.imshow(imsimobject, origin='lower')

Now that your simulation code is working, let's simulate a faint exoplanet.  Simulate an object at the same radius and position angle as before, but with dmag = 12.5.

In [ ]:
simcube = simulate_object(imcube, stellarim, pararr, 100., 60., 12.5)

Rerun PCA and display the resulting image:

In [ ]:
# add simulated object to data cube
imcubewithsimobject = imcube + simcube

# rerun PCA with 35 principal components.  You'll need to remake the covariance matrix in this case.


In [ ]:
# plot result


You can see that a planet 12.5 magnitudes fainter than the star is not too far off in appearance to the actual planets.  Let's first simulate some more planets at the separation of each of the actual planets and see if we can match the brightness of each planet in turn with a simulated version.  Let's start by simulating a Delta(mag)=12.5 planet at the separation of each of the 4 planets, but offset by 40 degrees in position angle.

In [ ]:
# Simulate 4 planets with delta(mag) = 12.5, 40 degrees offset from each actual planet



Re-run the PCA analysis, plot the resulting image

In [ ]:
# add simulated object to data cube

# rerun PCA with 35 principal components



In [ ]:
# plot result


Let's try simulating exoplanets with Delta(mag) values ranging from 11.5 to 13.1, in steps of 0.2 mag, in the same positions as in the last example, and save the results in a stack of images (like we did when testing out the number of principal components to retain).  

In [ ]:
# simulate exoplanets at a range of Delta(mag) values



In [ ]:
#plot results

fig = plt.figure(figsize=(10, 10*len(dmags)))

for i in np.arange(len(dmags)):

    ax = fig.add_subplot(len(dmags),1,i+1)

  

You can see by eye which of the simulated planets best match the actual observed planets.  With your project partner, go through the set of images you have plotted above and agree on the best matching Delta(mag) value for each planet.  HR 8799b is the furthest out of the planets, with HR 8799c, d, and e each successively closer in planet.

In [ ]:
planet_dmags = []

### Use your Delta(mag) estimates with an evolutionary model for exoplanets to estimate the mass of each planet

The absolute magnitude of HR 8799 (the star) in H band is: 2.22.  This is the magnitude of the star you would measure if you were a distance of 10 pc away from the star, and is essentially a measure of the star's intrinsic luminosity.  You've just found out how much fainter each planet is compared to the star in units of magnitudes.  Now calculate the absolute magnitude for each planet.  (Hint: this is just the Delta(mag) you've just found plus the absolute magnitude of the star.)

In [ ]:
# calculate the absolute magnitude of each planet


We'll now use the absolute magnitudes to estimate a mass for each planet from an evolutionary model.  This is a model that tracks the luminosity of brown dwarfs and giant exoplanets as they cools and dims with time.  Thus, if we know the absolute magnitude of a planet and also its age, we can estimate the mass that this corresponds to in the model.  You've just found the absolute magnitudes of each planet and the age of this star is about 50 million years.  The model is in a ascii table with two columns: mass (in Jupiter masses) and absolute H mag, Mh.

In [ ]:
model = Table.read('DUSTY_Hband_50Myr.txt', format='ascii')

In [ ]:
model

Use scipy.interpolate.interp1d to estimate the masses of each planet:

In [ ]:
f = interpolate.interp1d(model['Mh'], model['Mass'])

masses = f(planet_dmags + 2.22)

Print out your estimates for the masses of the planets.

In [ ]:
masses

How do the masses of the HR 8799 planets compare with the masses of gas giants in our own solar system?

# Change Log: please use the markdown cells at the end of the notebook to keep track of changes made by you and your project partner.